In [1]:
import pandas as pd
import networkx as nx
import itertools

In [2]:

file_path = 'Liguori_translations_FR_normalized_publishers_translators_places.xlsx'
df = pd.read_excel(file_path)

In [3]:
import math

In [4]:

G_1839_50 = nx.MultiDiGraph()

# Function to extract and sort years chronologically
def extract_years(row, cutoff_year0, cutoff_year1):
    year = row['First ed.']
    subsequent = row['Subsq. ed.']
    
    # Extract and filter years based on the cutoff year
    all_years = []
    
    if not math.isnan(year) and int(year) > cutoff_year0 and int(year) <= cutoff_year1:
    #if isinstance(year, (int, float)) and year < cutoff_year:
        all_years.append(int(year))
    
    if isinstance(subsequent, str):
        #subsequent_years = [int(x.split()[0]) for x in subsequent.split(',') if not (math.isnan(x) or not int(x.split()[0]) <= cutoff_year) else None]
        subsequent_years = [
            int(x.split()[0]) for x in subsequent.split(',')
            if x.strip() and not math.isnan(float(x.split()[0])) and cutoff_year0 < int(x.split()[0]) and int(x.split()[0]) <= cutoff_year1
            ]
        all_years.extend(subsequent_years)

    all_years = sorted(all_years)
    
    return all_years


def is_valid_year(x):
    try:
        year_int = int(x)
        return 1838 < year_int and year_int <= 1850
    except ValueError:
        return False
    

filtered_df = df[df['First ed.'].apply(lambda x: x if not (math.isnan(x) or not is_valid_year(x)) else None).notna()]

unique_publishers = set()

for publishers in filtered_df['Normalized Publisher']:
    if isinstance(publishers, str):
        unique_publishers.update([publisher.strip() for publisher in publishers.split(';')])
    else:
        unique_publishers.update([publishers])

for publisher in unique_publishers:
    G_1839_50.add_node(publisher, publications=[])
    

for index, row in filtered_df.iterrows():
    title = row['Title']
    years = extract_years(row, 1838, 1850)
    editions = sum(1 for year in years) if years != [] else 0  # Calculate number of editions based on years after 1825 and before 1839
    publishers = [publisher.strip() for publisher in row['Normalized Publisher'].split('; ')] if isinstance(row['Normalized Publisher'], str) else [row['Normalized Publisher']]
    translator = row['Normalized Translator']
    places = [place.strip() for place in row['Publ. place'].split('; ')] if isinstance(row['Publ. place'], str) else [str(row['Publ. place']).strip()]

    for publisher in unique_publishers:
        if publisher in publishers:
            G_1839_50.nodes[publisher]['publications'].append({
                'title': title,
                'years': years,
                'editions': [editions],
                'publisher': publishers,
                'translator': translator,
                'place': places
            })


for publisher in G_1839_50.nodes():
    if 'publications' in G_1839_50.nodes[publisher]:  # Check if 'publications' key exists
        magnitude = sum(pub['editions'][0] for pub in G_1839_50.nodes[publisher]['publications'])
        G_1839_50.nodes[publisher]['magnitude'] = magnitude

In [6]:

len(G_1839_50.nodes)

55

In [7]:

nodes_with_sizes = [(publisher, data['magnitude']) for publisher, data in G_1839_50.nodes(data=True) if 'magnitude' in data]

sorted_nodes = sorted(nodes_with_sizes, key=lambda x: x[1], reverse=True)

for publisher, size in sorted_nodes:
    print(f"Publisher: {publisher}, Size: {size}")

Publisher: landriot, Size: 20
Publisher: thibaud, Size: 20
Publisher: barbou, Size: 12
Publisher: mame, Size: 11
Publisher: plagaud, Size: 10
Publisher: gallienne, Size: 8
Publisher: librairie catholique, Size: 7
Publisher: casterman, Size: 6
Publisher: lesne, Size: 5
Publisher: lefort, Size: 5
Publisher: ardant, Size: 4
Publisher: perisse, Size: 3
Publisher: nan, Size: 3
Publisher: delsol, Size: 3
Publisher: société de saintvictor, Size: 3
Publisher: cornillac, Size: 3
Publisher: devillario, Size: 2
Publisher: vagner, Size: 2
Publisher: thomasmalvin, Size: 2
Publisher: parent-desbarres, Size: 2
Publisher: guyot, Size: 2
Publisher: prudhomme, Size: 2
Publisher: société des bons lecteurs, Size: 2
Publisher: pradel, Size: 1
Publisher: merson, Size: 1
Publisher: girard, Size: 1
Publisher: fr seguin, Size: 1
Publisher: bgat, Size: 1
Publisher: lagny, Size: 1
Publisher: thomas, Size: 1
Publisher: leprieur, Size: 1
Publisher: pinet, Size: 1
Publisher: paul mellier, Size: 1
Publisher: belin, 

In [8]:

def add_edge_with_attributes(G, publisher_A, publisher_B, title, source_years, target_years,
                             source_translator, target_translator, source_num_editions,
                             target_num_editions, source_place, target_place, weight, relation, key):
    G.add_edge(publisher_A, publisher_B, title=title, source_years=source_years, target_years=target_years,
               source_translator=source_translator, target_translator=target_translator,
               source_num_editions=source_num_editions, target_num_editions=target_num_editions,
               source_place=source_place, target_place=target_place, weight=weight, relation=relation, key=key)

In [9]:
existing_edges = set()

# Function to add edges between publishers with support for multiple edges of the same type
def add_edges_between_publishers(G):
    for pub_A, pub_B in itertools.combinations(G.nodes, 2):  # Iterate over pairs of nodes
        publications_A = G.nodes[pub_A]['publications']
        publications_B = G.nodes[pub_B]['publications']
        
        # Compare all publications between two publishers
        for publication_A in publications_A:
            for publication_B in publications_B:
                title_A, title_B = publication_A['title'], publication_B['title']
                years_A, years_B = publication_A['years'], publication_B['years']
                editions_A, editions_B = publication_A['editions'], publication_B['editions']
                translators_A, translators_B = publication_A['translator'], publication_B['translator']
                publishers_A, publishers_B = set(publication_A['publisher']), set(publication_B['publisher'])
                
                # Ensure translators_A and translators_B are lists
                translators_A = translators_A if isinstance(translators_A, list) else []
                translators_B = translators_B if isinstance(translators_B, list) else []

                
                # Initialize edge counters
                edge_counter = 0

                # Check for copublication
                if (len(publishers_A) == 1 & len(publishers_B) == 1) and title_A == title_B and set(years_A) == set(years_B):
                    # Create a unique edge key
                    edge_counter = sum(1 for key in existing_edges if f"copub_{pub_A}_{pub_B}_" in key) + 1
                    edge_key = f"copub_{pub_A}_{pub_B}_{title_A}_{edge_counter}"

                    # Check if the edge already exists
                    if edge_key not in existing_edges:
                        weight = max(editions_A[0], editions_B[0])
                        add_edge_with_attributes(
                            G, pub_A, pub_B, title_A, years_A, years_B, translators_A, translators_B,
                            editions_A, editions_B, publication_A['place'], publication_B['place'],
                            weight, 'copublication', edge_key
                        )
                        existing_edges.add(edge_key)

                # Check for reprint relation
                if set(translators_A) == set(translators_B) and title_A == title_B and years_A and years_B:
                    if len(years_A) > 0 and len(years_B) > 0:
                        if years_A[0] < years_B[0] and publishers_A != publishers_B:
                            edge_counter += 1
                            edge_key = f"reprint_{pub_A}_{pub_B}_{title_A}_{edge_counter}"

                            # Check if the edge already exists
                            if edge_key not in existing_edges:
                                weight = editions_B[0]
                                add_edge_with_attributes(
                                    G, pub_A, pub_B, title_A, years_A, years_B, translators_A, translators_B,
                                    editions_A, editions_B, publication_A['place'], publication_B['place'],
                                    weight, 'reprint', edge_key
                                )
                                existing_edges.add(edge_key)

                # Check for retranslation relation
                if set(translators_A) != set(translators_B) and title_A == title_B and years_A and years_B:
                    if len(years_A) > 0 and len(years_B) > 0:
                        if years_A[0] < years_B[0] and publishers_A != publishers_B:
                            edge_counter += 1
                            edge_key = f"retrans_{pub_A}_{pub_B}_{title_A}_{edge_counter}"

                            # Check if the edge already exists
                            if edge_key not in existing_edges:
                                weight = editions_B[0]
                                add_edge_with_attributes(
                                    G, pub_A, pub_B, title_A, years_A, years_B, translators_A, translators_B,
                                    editions_A, editions_B, publication_A['place'], publication_B['place'],
                                    weight, 'retranslation', edge_key
                                )
                                existing_edges.add(edge_key)

In [10]:

add_edges_between_publishers(G_1839_50)

In [11]:
len(G_1839_50.edges)

272

In [52]:
len(existing_edges) # G_thru_1825

471

In [16]:
len(existing_edges) # G_1826_38

550

In [12]:
len(existing_edges)

272

In [14]:

for title, first_ed, subsq_ed, edition, publisher, translator, place in zip(
        filtered_df['Title'],
        filtered_df['First ed.'],
        filtered_df['Subsq. ed.'],
        filtered_df['Editions'],
        filtered_df['Normalized Publisher'],
        filtered_df['Normalized Translator'],
        filtered_df['Publ. place']
    ):
    
    publisher_list = [pub.strip() for pub in str(publisher).split(';') if pub.strip()]
    
    translator_list = [trans.strip() for trans in str(translator).split(';') if trans.strip()]
    
    place_list = [pl.strip() for pl in str(place).split(';') if pl.strip()]
    
    if len(publisher_list) > 1:
        
        for pub_A, pub_B in itertools.combinations(publisher_list, 2):
            # Assign a weight based on editions
            weight = max(edition, edition)
            # Generate a unique edge key for each copublication instance
            edge_counter = sum(1 for key in existing_edges if f"copub_{pub_A}_{pub_B}_" in key) + 1
            edge_key = f"copub_{pub_A}_{pub_B}_{title}_{edge_counter}"

            # Check if the edge already exists
            if edge_key not in existing_edges:
                add_edge_with_attributes(
                    G_1839_50, pub_A, pub_B, title, [first_ed, subsq_ed], [first_ed, subsq_ed], translator_list, translator_list,
                    [edition], [edition], place_list, place_list, weight, 'copublication', edge_key
                )
                existing_edges.add(edge_key)


In [15]:

out_degrees = dict(G_1839_50.out_degree())
in_degrees = dict(G_1839_50.in_degree())
overall_degrees = {node: out_degrees.get(node, 0) + in_degrees.get(node, 0) for node in G_1839_50.nodes()}

nodes_by_out_degree = sorted(out_degrees, key=out_degrees.get, reverse=True)
nodes_by_in_degree = sorted(in_degrees, key=in_degrees.get, reverse=True)
nodes_by_overall_degree = sorted(overall_degrees, key=overall_degrees.get, reverse=True)

print("Nodes ordered by outgoing degree:")
for node in nodes_by_out_degree:
    print(node, out_degrees[node])

print("\nNodes ordered by incoming degree:")
for node in nodes_by_in_degree:
    print(node, in_degrees[node])

print("\nNodes ordered by overall degree:")
for node in nodes_by_overall_degree:
    print(node, overall_degrees[node])

Nodes ordered by outgoing degree:
mame 28
devillario 22
plagaud 21
pradel 20
gallienne 20
landriot 20
fr seguin 19
thibaud 19
pinet 18
delsol 12
belin 10
barbou 9
leprieur 9
royer 8
lesne 8
de lamarzelle 7
pornin 6
thomasmalvin 5
aubanel 5
dieulafoy 4
merson 3
guyot 3
pellerin 3
thomas 2
paul mellier 2
villetcollignon 2
prudhomme 2
cornillac 2
girard 1
caron 1
tubergue 1
abadie 1
parent-desbarres 1
librairie catholique 1
vagner 0
perisse 0
bgat 0
lagny 0
nan 0
casterman 0
vanderborght 0
desmoulins 0
mellier 0
ardent 0
lambert 0
poussielgue 0
jacquot 0
société de saintvictor 0
waille 0
heger 0
société des bons lecteurs 0
ardant 0
bureau des mlanges religieux 0
hanicq 0
lefort 0

Nodes ordered by incoming degree:
ardant 31
landriot 25
lefort 24
thibaud 23
cornillac 20
delsol 15
lesne 15
pellerin 14
abadie 13
villetcollignon 12
librairie catholique 12
pornin 9
aubanel 8
dieulafoy 8
de lamarzelle 8
prudhomme 7
thomas 6
leprieur 5
belin 5
plagaud 5
ardent 5
gallienne 4
guyot 4
société de sa

In [17]:


weights = []

outgoing_weighted_degrees = {}
incoming_weighted_degrees = {}
overall_weighted_degrees = {}

for node in G_1839_50.nodes():
    successors = []
    for node_id in G_1839_50.successors(node):
        successors.append(node_id)
    weights = []
    for u, v, key, attr in G_1839_50.edges(keys=True, data=True):
        if (u in successors and v == node) or (u == node and v in successors):
            weight = attr['weight']
            #weights.append(sum(weight))
            weights.append(weight)
    #outgoing_weight = sum(sum(G11.edges[node, neighbor]['weight']) for neighbor in G11.successors(node))
    outgoing_weighted_degrees[node] = sum(weights)

for node in G_1839_50.nodes():
    predecessors = []
    for node_id in G_1839_50.predecessors(node):
        predecessors.append(node_id)
    weights = []
    for u, v, key, attr in G_1839_50.edges(keys=True, data=True):
        if (u in predecessors and v == node) or (u == node and v in predecessors):
            weight = attr['weight']
            # weights.append(sum(weight))
            weights.append(weight)
    #incoming_weight = sum(sum(G11.edges[neighbor, node]['weight']) for neighbor in G11.predecessors(node))
    incoming_weighted_degrees[node] = sum(weights)

for node in G_1839_50.nodes():
    overall_weighted_degree = outgoing_weighted_degrees.get(node, 0) + incoming_weighted_degrees.get(node, 0)
    overall_weighted_degrees[node] = overall_weighted_degree

sorted_outgoing_weighted_degrees = dict(sorted(outgoing_weighted_degrees.items(), key=lambda item: item[1], reverse=True))
sorted_incoming_weighted_degrees = dict(sorted(incoming_weighted_degrees.items(), key=lambda item: item[1], reverse=True))
sorted_overall_weighted_degrees = dict(sorted(overall_weighted_degrees.items(), key=lambda item: item[1], reverse=True))

print("Outgoing Weighted Degrees:", sorted_outgoing_weighted_degrees)
print("Incoming Weighted Degrees:", sorted_incoming_weighted_degrees)
print("Overall Weighted Degrees:", sorted_overall_weighted_degrees)

Outgoing Weighted Degrees: {'thibaud': 36, 'mame': 35, 'devillario': 29, 'pradel': 29, 'plagaud': 26, 'fr seguin': 23, 'gallienne': 23, 'landriot': 23, 'pinet': 20, 'delsol': 18, 'royer': 12, 'lesne': 12, 'thomasmalvin': 11, 'belin': 11, 'barbou': 10, 'leprieur': 10, 'de lamarzelle': 8, 'pornin': 7, 'aubanel': 6, 'dieulafoy': 5, 'pellerin': 4, 'merson': 3, 'villetcollignon': 3, 'guyot': 3, 'cornillac': 3, 'thomas': 2, 'paul mellier': 2, 'prudhomme': 2, 'girard': 1, 'caron': 1, 'tubergue': 1, 'abadie': 1, 'parent-desbarres': 1, 'librairie catholique': 1, 'vagner': 0, 'perisse': 0, 'bgat': 0, 'lagny': 0, nan: 0, 'casterman': 0, 'vanderborght': 0, 'desmoulins': 0, 'mellier': 0, 'ardent': 0, 'lambert': 0, 'poussielgue': 0, 'jacquot': 0, 'société de saintvictor': 0, 'waille': 0, 'heger': 0, 'société des bons lecteurs': 0, 'ardant': 0, 'bureau des mlanges religieux': 0, 'hanicq': 0, 'lefort': 0}
Incoming Weighted Degrees: {'ardant': 53, 'landriot': 42, 'librairie catholique': 32, 'cornillac'

In [18]:

from collections import Counter


source_place_counter = Counter()
target_place_counter = Counter()
overall_place_counter = Counter()


for publisher_A, publisher_B, data in G_1839_50.edges(data=True):
    if isinstance(data['source_place'], list):
        source_places = data['source_place']
    else:
        source_places = [data['source_place']]
    if isinstance(data['target_place'], list):
        target_places = data['target_place']
    else:
        target_places = [data['target_place']]

    
    for i, place_list in enumerate(source_places):
        if isinstance(place_list, str):
            source_places[i] = [place_list]
    
    for i, place_list in enumerate(target_places):
        if isinstance(place_list, str):
            #place_list = [place_list]
            target_places[i] = [place_list]
            
    for place_list in source_places:
        for place in place_list:
            source_place_counter[place] += 1
            
    for place_list in target_places:
        for place in place_list:
            target_place_counter[place] += 1
    
    for place_list in source_places + target_places:
        #if isinstance(place_list, str):
            #place_list = [place_list]  # Convert string to list
        for place in place_list:
            overall_place_counter[place] += 1


sorted_source_places = source_place_counter.most_common()
sorted_target_places = target_place_counter.most_common()
sorted_overall_places = overall_place_counter.most_common()


print("Sorted Source Places:", sorted_source_places)
print("Sorted Target Places:", sorted_target_places)
print("Sorted Overall Places:", sorted_overall_places)

Sorted Source Places: [('Paris', 48), ('Clermont', 40), ('Toulouse', 36), ('Lyon', 33), ('Avignon', 24), ('Carpentras', 22), ('Le Mans', 20), ('Nevers', 18), ('Tours', 16), ('Limoges', 9), ('Vannes', 7), ('Sens', 5), ('Nantes', 3), ('Épinal', 3), ('Nancy', 2), ('Verdun', 2), ('Saint Brieuc', 2), ('Châtillon', 2), ('Amiens', 1), ('Besançon', 1), ('Saint-Gaudens', 1)]
Sorted Target Places: [('Clermont', 60), ('Limoges', 33), ('Toulouse', 25), ('Lyon', 25), ('Lille', 24), ('Paris', 21), ('Châtillon', 20), ('Épinal', 14), ('Saint-Gaudens', 13), ('Verdun', 12), ('Tours', 10), ('Avignon', 9), ('Vannes', 8), ('Saint Brieuc', 7), ('Nancy', 6), ('Le Mans', 4), ('Plancy', 3), ('Amiens', 1), ('Besançon', 1)]
Sorted Overall Places: [('Clermont', 100), ('Paris', 69), ('Toulouse', 61), ('Lyon', 58), ('Limoges', 42), ('Avignon', 33), ('Tours', 26), ('Le Mans', 24), ('Lille', 24), ('Carpentras', 22), ('Châtillon', 22), ('Nevers', 18), ('Épinal', 17), ('Vannes', 15), ('Saint-Gaudens', 14), ('Verdun', 1

In [19]:

source_place_counter_weighted = Counter()
target_place_counter_weighted = Counter()
overall_place_counter_weighted = Counter()

for publisher_A, publisher_B, data in G_1839_50.edges(data=True):
    # Get the weight of the current edge
    weight = data['weight']
    
    if isinstance(data['source_place'], list):
        source_places = data['source_place']
    else:
        source_places = [data['source_place']]
    if isinstance(data['target_place'], list):
        target_places = data['target_place']
    else:
        target_places = [data['target_place']]

    for i, place_list in enumerate(source_places):
        if isinstance(place_list, str):
            source_places[i] = [place_list]

    for i, place_list in enumerate(target_places):
        if isinstance(place_list, str):
            target_places[i] = [place_list]

    for place_list in source_places:
        for place in place_list:
            # source_place_counter_weighted[place] += weight[0]
            source_place_counter_weighted[place] += weight
            
    for place_list in target_places:
        for place in place_list:
            # target_place_counter_weighted[place] += weight[0]
            target_place_counter_weighted[place] += weight

    for place_list in source_places + target_places:
        for place in place_list:
            # overall_place_counter_weighted[place] += weight[0]
            overall_place_counter_weighted[place] += weight

sorted_source_places_weighted = source_place_counter_weighted.most_common()
sorted_target_places_weighted = target_place_counter_weighted.most_common()
sorted_overall_places_weighted = overall_place_counter_weighted.most_common()

print("Sorted Source Places (Weighted):", sorted_source_places_weighted)
print("Sorted Target Places (Weighted):", sorted_target_places_weighted)
print("Sorted Overall Places (Weighted):", sorted_overall_places_weighted)

Sorted Source Places (Weighted): [('Clermont', 60), ('Paris', 58), ('Toulouse', 49), ('Lyon', 42), ('Carpentras', 29), ('Avignon', 29), ('Le Mans', 23), ('Tours', 20), ('Nevers', 20), ('Sens', 11), ('Limoges', 10), ('Vannes', 8), ('Épinal', 4), ('Nantes', 3), ('Verdun', 3), ('Châtillon', 3), ('Nancy', 2), ('Saint Brieuc', 2), ('Amiens', 1), ('Besançon', 1), ('Saint-Gaudens', 1)]
Sorted Target Places (Weighted): [('Clermont', 100), ('Limoges', 61), ('Lyon', 29), ('Châtillon', 29), ('Toulouse', 26), ('Lille', 24), ('Paris', 21), ('Épinal', 14), ('Saint-Gaudens', 13), ('Verdun', 12), ('Tours', 12), ('Avignon', 9), ('Vannes', 8), ('Saint Brieuc', 7), ('Nancy', 6), ('Le Mans', 4), ('Plancy', 3), ('Amiens', 1), ('Besançon', 1)]
Sorted Overall Places (Weighted): [('Clermont', 160), ('Paris', 79), ('Toulouse', 75), ('Limoges', 71), ('Lyon', 71), ('Avignon', 38), ('Tours', 32), ('Châtillon', 32), ('Carpentras', 29), ('Le Mans', 27), ('Lille', 24), ('Nevers', 20), ('Épinal', 18), ('Vannes', 16),

In [21]:

# Initialize counters for unweighted and weighted titles
unweighted_title_counter = Counter()
weighted_title_counter = Counter()

# Initialize dictionaries to store concatenated source and target places for each title
title_source_places = {}
title_target_places = {}

# Iterate over edges in the graph
for publisher_A, publisher_B, data in G_1839_50.edges(data=True):
    title = data['title']
    weight = data['weight']
    
    # Increment the unweighted and weighted title counters
    unweighted_title_counter[title] += 1
    weighted_title_counter[title] += weight
    
    if isinstance(data['source_place'], list):
        source_places = data['source_place']
    else:
        source_places = [data['source_place']]
    if isinstance(data['target_place'], list):
        target_places = data['target_place']
    else:
        target_places = [data['target_place']]

    for i, place_list in enumerate(source_places):
        if isinstance(place_list, str):
            source_places[i] = [place_list]

    for i, place_list in enumerate(target_places):
        if isinstance(place_list, str):
            target_places[i] = [place_list]
    
    if title not in title_source_places:
                title_source_places[title] = [] 
            
    for place_list in source_places:
        for place in place_list:
            title_source_places[title].append(place)
    
    if title not in title_target_places:
                title_target_places[title] = [] 
            
    for place_list in target_places:
        for place in place_list:          
            title_target_places[title].append(place)

# Sort titles based on unweighted occurrences
sorted_unweighted_titles = unweighted_title_counter.most_common()

# Sort titles based on weighted occurrences
sorted_weighted_titles = weighted_title_counter.most_common()

for elem in sorted_unweighted_titles:
    print(f"Title and number of connections between places it occasioned: {elem} | Source Places: {set(title_source_places[elem[0]])} | Target Places: {set(title_target_places[elem[0]])}")

for ele in sorted_weighted_titles:
    print(f"Title and weighted number of connections between places it occasioned: {ele} | Source Places: {set(title_source_places[ele[0]])} | Target Places: {set(title_target_places[ele[0]])}")

Title and number of connections between places it occasioned: ('Visite au S. Sacrement et à\xa0 la Sainte Vierge', 223) | Source Places: {'Paris', 'Verdun', 'Toulouse', 'Limoges', 'Le Mans', 'Nancy', 'Carpentras', 'Avignon', 'Vannes', 'Nevers', 'Tours', 'Châtillon', 'Saint-Gaudens', 'Lyon', 'Épinal', 'Clermont'} | Target Places: {'Paris', 'Lille', 'Verdun', 'Toulouse', 'Lyon', 'Limoges', 'Le Mans', 'Nancy', 'Avignon', 'Vannes', 'Châtillon', 'Saint-Gaudens', 'Tours', 'Épinal', 'Clermont'}
Title and number of connections between places it occasioned: ('Pratique de l\'amour envers Jésus-Christ, tirée des paroles de S. Paul : "Caritas patiens est, benigna est"... par le B. Alphonse de Liguori', 24) | Source Places: {'Limoges', 'Le Mans', 'Saint Brieuc', 'Tours', 'Lyon', 'Clermont'} | Target Places: {'Lyon', 'Limoges', 'Saint Brieuc', 'Clermont'}
Title and number of connections between places it occasioned: ('Paraphrase du "Salve Regina", par le bienheureux Alphonse de Liguori...', 17) | So

In [22]:


G_combined = nx.DiGraph()

for u, v, key, attr in G_1839_50.edges(keys=True, data=True):
    weight = attr.get('weight', 0)  # Default to 0 if 'weight' is missing

    if G_combined.has_edge(u, v):
        G_combined[u][v]['weight'] += weight
    else:
        G_combined.add_edge(u, v, weight=weight)

for node, data in G_1839_50.nodes(data=True):
    if not G_combined.has_node(node):
        G_combined.add_node(node, **data)

In [23]:

len(G_combined.nodes)

55

In [79]:
len(G_combined.edges) # out of the 471 in G_thru_1825 now collapsed into a digraph:

460

In [24]:
len(G_combined.edges) # out of 550 edges in G_1826_38 

529

In [25]:
len(G_combined.edges) # out of 272 edges in G_1839_50

257

In [26]:

closeness_centralities = nx.closeness_centrality(G_combined)

betweenness_centralities = nx.betweenness_centrality(G_combined)

In [27]:

eigenvector_centralities = nx.eigenvector_centrality(G_combined)

In [85]:
# eigenvector_centralities = nx.eigenvector_centrality(G_combined, max_iter = 500)

In [28]:
import operator

In [29]:

sorted_closeness = sorted(closeness_centralities.items(), key=operator.itemgetter(1), reverse=True)
sorted_betweenness = sorted(betweenness_centralities.items(), key=operator.itemgetter(1), reverse=True)
sorted_eigenvector = sorted(eigenvector_centralities.items(), key=operator.itemgetter(1), reverse=True)

In [30]:
sorted_closeness

[('ardant', 0.45806100217864926),
 ('lefort', 0.4273504273504274),
 ('cornillac', 0.38571428571428573),
 ('thibaud', 0.35555555555555557),
 ('pellerin', 0.30458089668615984),
 ('librairie catholique', 0.28935185185185186),
 ('landriot', 0.26016260162601623),
 ('lesne', 0.26016260162601623),
 ('delsol', 0.25396825396825395),
 ('villetcollignon', 0.25161030595813205),
 ('abadie', 0.23703703703703702),
 ('pornin', 0.23703703703703702),
 ('dieulafoy', 0.22695035460992904),
 ('de lamarzelle', 0.22695035460992904),
 ('prudhomme', 0.20667989417989419),
 ('ardent', 0.19290123456790123),
 ('société de saintvictor', 0.18371546149323925),
 ('plagaud', 0.1807909604519774),
 ('pradel', 0.16666666666666666),
 ('guyot', 0.1616161616161616),
 ('aubanel', 0.14814814814814814),
 ('thomas', 0.13852813852813853),
 ('leprieur', 0.13852813852813853),
 ('mellier', 0.13778659611992947),
 ('belin', 0.13675213675213677),
 ('gallienne', 0.1316872427983539),
 ('barbou', 0.12549019607843137),
 ('mame', 0.104575163

In [31]:

sorted_betweenness 

[('landriot', 0.08556953179594688),
 ('delsol', 0.07299091544374563),
 ('thibaud', 0.0590146750524109),
 ('pradel', 0.052119729792685775),
 ('gallienne', 0.017353831819240623),
 ('plagaud', 0.013475425110645235),
 ('guyot', 0.009433962264150943),
 ('mame', 0.0015373864430468205),
 ('barbou', 0.0013976240391334728),
 ('cornillac', 0.0005241090146750524),
 ('prudhomme', 0.00046587467971115766),
 ('lesne', 0.00040764034474726293),
 ('belin', 0.00034940600978336826),
 ('dieulafoy', 5.823433496389471e-05),
 ('de lamarzelle', 5.823433496389471e-05),
 ('pornin', 5.823433496389471e-05),
 ('devillario', 0.0),
 ('fr seguin', 0.0),
 ('thomas', 0.0),
 ('leprieur', 0.0),
 ('aubanel', 0.0),
 ('abadie', 0.0),
 ('villetcollignon', 0.0),
 ('pellerin', 0.0),
 ('ardant', 0.0),
 ('librairie catholique', 0.0),
 ('lefort', 0.0),
 ('ardent', 0.0),
 ('société de saintvictor', 0.0),
 ('merson', 0.0),
 (nan, 0.0),
 ('girard', 0.0),
 ('pinet', 0.0),
 ('royer', 0.0),
 ('paul mellier', 0.0),
 ('parent-desbarres', 

In [32]:
sorted_eigenvector

[('lefort', 0.562705272275513),
 ('ardant', 0.5507787147017774),
 ('cornillac', 0.40064489075572035),
 ('pellerin', 0.24594106945329605),
 ('thibaud', 0.19854405261300956),
 ('librairie catholique', 0.18710701223417717),
 ('lesne', 0.1391088888782531),
 ('pornin', 0.10567383532347875),
 ('villetcollignon', 0.10009506526669795),
 ('delsol', 0.09905880698586582),
 ('landriot', 0.08640125657736586),
 ('dieulafoy', 0.08027483165866901),
 ('de lamarzelle', 0.08027483165866901),
 ('abadie', 0.06737160993550867),
 ('prudhomme', 0.04814676900107184),
 ('ardent', 0.04628313138788746),
 ('plagaud', 0.032775202484870404),
 ('pradel', 0.031343269908818784),
 ('société de saintvictor', 0.030475111241149006),
 ('aubanel', 0.023582615652896804),
 ('thomas', 0.01849347434849708),
 ('guyot', 0.01580802014682505),
 ('leprieur', 0.014776173846178126),
 ('gallienne', 0.013055380318262231),
 ('belin', 0.01122452717875739),
 ('mellier', 0.01036992611654791),
 ('barbou', 0.009917377531283546),
 ('mame', 0.00

In [33]:

weighted_closeness = nx.closeness_centrality(G_combined, distance='weight')
weighted_betweenness = nx.betweenness_centrality(G_combined, weight='weight')
weighted_eigenvector = nx.eigenvector_centrality(G_combined, max_iter = 500, weight='weight')

sorted_weighted_closeness = sorted(weighted_closeness.items(), key=lambda x: x[1], reverse=True)
sorted_weighted_betweenness = sorted(weighted_betweenness.items(), key=lambda x: x[1], reverse=True)
sorted_weighted_eigenvector = sorted(weighted_eigenvector.items(), key=lambda x: x[1], reverse=True)

In [34]:

sorted_weighted_closeness

[('lefort', 0.39682539682539686),
 ('thibaud', 0.2882882882882883),
 ('pellerin', 0.27557319223985893),
 ('cornillac', 0.2755102040816326),
 ('ardant', 0.24720752498530277),
 ('librairie catholique', 0.12580515297906603),
 ('lesne', 0.06971677559912853),
 ('pornin', 0.05360134003350083),
 ('prudhomme', 0.052371375900787664),
 ('guyot', 0.047619047619047616),
 ('dieulafoy', 0.0471976401179941),
 ('de lamarzelle', 0.0471976401179941),
 ('villetcollignon', 0.0462962962962963),
 ('ardent', 0.04318684355997789),
 ('delsol', 0.043184885290148446),
 ('société de saintvictor', 0.042551742919389984),
 ('abadie', 0.042328042328042326),
 ('landriot', 0.04199475065616798),
 ('plagaud', 0.03965303593556381),
 ('mellier', 0.039367598891408416),
 ('lagny', 0.037037037037037035),
 ('aubanel', 0.036655211912943866),
 ('pradel', 0.0365296803652968),
 ('thomas', 0.03567447045707915),
 ('leprieur', 0.035555555555555556),
 ('belin', 0.03532008830022075),
 ('gallienne', 0.034972677595628415),
 ('mame', 0.02

In [35]:

sorted_weighted_betweenness

[('landriot', 0.08427544386663882),
 ('delsol', 0.07542083760637218),
 ('pradel', 0.05625436757512229),
 ('thibaud', 0.054949540326898805),
 ('gallienne', 0.01971232238527836),
 ('plagaud', 0.017222426420539626),
 ('guyot', 0.010414114187699093),
 ('prudhomme', 0.0018022392236228713),
 ('de lamarzelle', 0.0014829209535708488),
 ('pornin', 0.0014829209535708488),
 ('belin', 0.0012592481670049803),
 ('cornillac', 0.0009459297981310558),
 ('pellerin', 0.0009073842145120972),
 ('leprieur', 0.0008749015562432753),
 ('mame', 0.0006988120195667365),
 ('dieulafoy', 0.00069675743155827),
 ('aubanel', 0.0005978725056293191),
 ('lesne', 0.00046587467971115766),
 ('thomas', 0.0004367575122292103),
 ('villetcollignon', 0.0004258165159632665),
 ('devillario', 0.0),
 ('barbou', 0.0),
 ('fr seguin', 0.0),
 ('abadie', 0.0),
 ('ardant', 0.0),
 ('librairie catholique', 0.0),
 ('lefort', 0.0),
 ('ardent', 0.0),
 ('société de saintvictor', 0.0),
 ('merson', 0.0),
 (nan, 0.0),
 ('girard', 0.0),
 ('pinet', 0

In [36]:
sorted_weighted_eigenvector 

[('ardant', 0.5690725027009835),
 ('landriot', 0.43633061749650864),
 ('librairie catholique', 0.37456751897078716),
 ('lefort', 0.36086130913187764),
 ('cornillac', 0.2460744635552106),
 ('lesne', 0.19687144786400265),
 ('pellerin', 0.1749246009591842),
 ('prudhomme', 0.12794377977507482),
 ('thibaud', 0.12275266400986226),
 ('pornin', 0.09279270695979921),
 ('delsol', 0.08476640872641146),
 ('villetcollignon', 0.08403611421621995),
 ('dieulafoy', 0.08177028203894332),
 ('de lamarzelle', 0.08177028203894332),
 ('ardent', 0.0695635858059519),
 ('abadie', 0.06748169418162801),
 ('plagaud', 0.060747909891418185),
 ('société de saintvictor', 0.06047743470592447),
 ('pradel', 0.022852560076699252),
 ('barbou', 0.012322519692087398),
 ('guyot', 0.009086151100027445),
 ('mellier', 0.008188516474673883),
 ('thomas', 0.005639316872457459),
 ('aubanel', 0.004939149283548453),
 ('gallienne', 0.004741682247103898),
 ('leprieur', 0.0037892098304995645),
 ('belin', 0.003339103144077401),
 ('mame', 